In [1]:
pip install praw nltk spacy rake-nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 3.7 MB/s eta 0:00:00


In [2]:
pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 20.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 9.8 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039190 sha256=2f8779d98ae38ec14eea9b85a1ce4ee7cc8da49d237ac916854b2229ca08feea
  Stored in dir

In [3]:
pip install pandas yake rake-nltk summa keybert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.3 MB/s eta 0:00:00
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54388 sha256=c337dbdf5edec871c957358d46c6a6bfdba6c109549606e001ed578a3f26b242
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39124 sha256=0fca626b188647bde08a7708528c67923e6100f3f30773e13d7a27b59c3696dc
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
Successfully built summa keybert


In [4]:
import pandas as pd
import numpy as np
import re
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from rake_nltk import Rake
from collections import Counter
#tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
#wordvec
from gensim.models import Word2Vec
#kemans
from sklearn.cluster import KMeans
#LDA
from sklearn.decomposition import LatentDirichletAllocation

In [5]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
bc_whole_df = pd.read_csv("/content/breast_cancer_merged.csv")
bc_whole_df.head()

,id,name,title,selftext,num_comments,score_x,score_y,body,controversiality
0,nrcy0,t3_nrcy0,I'm cycling 4000 miles this summer to raise mo...,I'm doing this mostly for my mom who was diagn...,2,3.0,2,I love this idea. You should repost this in a...,0
1,qls70,t3_qls70,How did your hair grow back after chemotherapy?,I am just curious.... what happened to your ha...,3,2.0,2,"Mine was originally thick, wavy and red-brown....",0
2,qls70,t3_qls70,How did your hair grow back after chemotherapy?,I am just curious.... what happened to your ha...,3,2.0,2,"My mom always had coarse, thick, wavy hair. A...",0
3,qnpht,t3_qnpht,33yo Male...,"Hey everyone, I'm a 33yo male, and my left nip...",10,2.0,5,"Honestly, the internet was my worst enemy at t...",0
4,qnpht,t3_qnpht,33yo Male...,"Hey everyone, I'm a 33yo male, and my left nip...",10,2.0,1,"Breast cancer in men is quite common, more tha...",0


In [7]:
import pandas as pd
import re

# Load your DataFrame
bc_whole_df = pd.read_csv("/content/breast_cancer_merged.csv")

# Define a function to extract URLs and return them as a single string
def extract_urls(text):
    # Regular expression pattern for matching URLs
    url_pattern = r'https?://\S+|www\.\S+'
    found_urls = re.findall(url_pattern, text)
    # Join the URLs into a single string, separated by commas
    return ', '.join(found_urls)

# Apply the function to the selftext column
bc_whole_df['urls'] = bc_whole_df['selftext'].apply(lambda x: extract_urls(str(x)))

# Display the first few rows of the DataFrame to verify
print(bc_whole_df.head())


      id      name                                              title  \
0  nrcy0  t3_nrcy0  I'm cycling 4000 miles this summer to raise mo...   
1  qls70  t3_qls70    How did your hair grow back after chemotherapy?   
2  qls70  t3_qls70    How did your hair grow back after chemotherapy?   
3  qnpht  t3_qnpht                                       33yo Male...   
4  qnpht  t3_qnpht                                       33yo Male...   

                                            selftext  num_comments  score_x  \
0  I'm doing this mostly for my mom who was diagn...             2      3.0   
1  I am just curious.... what happened to your ha...             3      2.0   
2  I am just curious.... what happened to your ha...             3      2.0   
3  Hey everyone, I'm a 33yo male, and my left nip...            10      2.0   
4  Hey everyone, I'm a 33yo male, and my left nip...            10      2.0   

   score_y                                               body  \
0        2  I love th

In [8]:
bc_whole_df

,id,name,title,selftext,num_comments,score_x,score_y,body,controversiality,urls
0,nrcy0,t3_nrcy0,I'm cycling 4000 miles this summer to raise mo...,I'm doing this mostly for my mom who was diagn...,2,3.0,2,I love this idea. You should repost this in a...,0,http://www.cancer.org/Treatment/SupportProgram...
1,qls70,t3_qls70,How did your hair grow back after chemotherapy?,I am just curious.... what happened to your ha...,3,2.0,2,"Mine was originally thick, wavy and red-brown....",0,
2,qls70,t3_qls70,How did your hair grow back after chemotherapy?,I am just curious.... what happened to your ha...,3,2.0,2,"My mom always had coarse, thick, wavy hair. A...",0,
3,qnpht,t3_qnpht,33yo Male...,"Hey everyone, I'm a 33yo male, and my left nip...",10,2.0,5,"Honestly, the internet was my worst enemy at t...",0,
4,qnpht,t3_qnpht,33yo Male...,"Hey everyone, I'm a 33yo male, and my left nip...",10,2.0,1,"Breast cancer in men is quite common, more tha...",0,
...,...,...,...,...,...,...,...,...,...,...
65547,zzym9y,t3_zzym9y,For those who did delayed bilateral reconstruc...,Hi - new poster (long time lurker) here. I am ...,67,10.0,2,I had my mastectomy in June of 2021 and Diep i...,0,
65548,zzym9y,t3_zzym9y,For those who did delayed bilateral reconstruc...,Hi - new poster (long time lurker) here. I am ...,67,10.0,3,I have Natrelle Inspira “gummy bear” implants....,0,
65549,zzym9y,t3_zzym9y,For those who did delayed bilateral reconstruc...,Hi - new poster (long time lurker) here. I am ...,67,10.0,4,I had a double mastectomy with tissue expander...,0,
65550,zzym9y,t3_zzym9y,For those who did delayed bilateral reconstruc...,Hi - new poster (long time lurker) here. I am ...,67,10.0,2,My story is similar to yours. I had a right ma...,0,


In [9]:
columns_to_include = ['title','selftext','body','score_x','urls']
question_df = bc_whole_df[columns_to_include]
len(question_df)

65552

In [10]:
question_df = question_df.drop_duplicates(keep='first')
len(question_df)

65411

In [11]:
question_df.head()

,title,selftext,body,score_x,urls
0,I'm cycling 4000 miles this summer to raise mo...,I'm doing this mostly for my mom who was diagn...,I love this idea. You should repost this in a...,3.0,http://www.cancer.org/Treatment/SupportProgram...
1,How did your hair grow back after chemotherapy?,I am just curious.... what happened to your ha...,"Mine was originally thick, wavy and red-brown....",2.0,
2,How did your hair grow back after chemotherapy?,I am just curious.... what happened to your ha...,"My mom always had coarse, thick, wavy hair. A...",2.0,
3,33yo Male...,"Hey everyone, I'm a 33yo male, and my left nip...","Honestly, the internet was my worst enemy at t...",2.0,
4,33yo Male...,"Hey everyone, I'm a 33yo male, and my left nip...","Breast cancer in men is quite common, more tha...",2.0,


In [12]:
def clean_text(text):
    # Lowercasing
    text = text.lower()

    # Removing URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Removing user @ references and '#' from tweet
    text = re.sub(r'\@\w+|\#','', text)

    # Removing punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenization
    tokens = nltk.word_tokenize(text)

    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

    return " ".join(lemmatized_tokens)

In [13]:
#data cleaning
question_df['cleaned_question'] = question_df['selftext'].apply(clean_text)

In [14]:
question_df['cleaned_question'].head(5)

0    im mostly mom diagnosed breast cancer septembe...
1    curious happened hair chemotherapy mine stick ...
2    curious happened hair chemotherapy mine stick ...
3    hey everyone im 33yo male left nipple felt irr...
4    hey everyone im 33yo male left nipple felt irr...
Name: cleaned_question, dtype: object

In [15]:
words_to_remove = ['hi', 'hey']

def remove_short(text, words_to_remove):
    words = text.split()
    filtered_words = [word for word in words if word not in words_to_remove]
    return ' '.join(filtered_words)

In [16]:
question_df['cleaned_question'] = question_df['cleaned_question'].apply(lambda x: remove_short(x, words_to_remove))

In [17]:
question_df['cleaned_question'].head(5)

0    im mostly mom diagnosed breast cancer septembe...
1    curious happened hair chemotherapy mine stick ...
2    curious happened hair chemotherapy mine stick ...
3    everyone im 33yo male left nipple felt irritat...
4    everyone im 33yo male left nipple felt irritat...
Name: cleaned_question, dtype: object

In [18]:
question_df

,title,selftext,body,score_x,urls,cleaned_question
0,I'm cycling 4000 miles this summer to raise mo...,I'm doing this mostly for my mom who was diagn...,I love this idea. You should repost this in a...,3.0,http://www.cancer.org/Treatment/SupportProgram...,im mostly mom diagnosed breast cancer septembe...
1,How did your hair grow back after chemotherapy?,I am just curious.... what happened to your ha...,"Mine was originally thick, wavy and red-brown....",2.0,,curious happened hair chemotherapy mine stick ...
2,How did your hair grow back after chemotherapy?,I am just curious.... what happened to your ha...,"My mom always had coarse, thick, wavy hair. A...",2.0,,curious happened hair chemotherapy mine stick ...
3,33yo Male...,"Hey everyone, I'm a 33yo male, and my left nip...","Honestly, the internet was my worst enemy at t...",2.0,,everyone im 33yo male left nipple felt irritat...
4,33yo Male...,"Hey everyone, I'm a 33yo male, and my left nip...","Breast cancer in men is quite common, more tha...",2.0,,everyone im 33yo male left nipple felt irritat...
...,...,...,...,...,...,...
65547,For those who did delayed bilateral reconstruc...,Hi - new poster (long time lurker) here. I am ...,I had my mastectomy in June of 2021 and Diep i...,10.0,,new poster long time lurker 42 diagnosed febru...
65548,For those who did delayed bilateral reconstruc...,Hi - new poster (long time lurker) here. I am ...,I have Natrelle Inspira “gummy bear” implants....,10.0,,new poster long time lurker 42 diagnosed febru...
65549,For those who did delayed bilateral reconstruc...,Hi - new poster (long time lurker) here. I am ...,I had a double mastectomy with tissue expander...,10.0,,new poster long time lurker 42 diagnosed febru...
65550,For those who did delayed bilateral reconstruc...,Hi - new poster (long time lurker) here. I am ...,My story is similar to yours. I had a right ma...,10.0,,new poster long time lurker 42 diagnosed febru...


In [19]:
tfidf_vec = TfidfVectorizer(max_df = 0.95, min_df= 2)

tfidf_ques = tfidf_vec.fit_transform(question_df['cleaned_question'])

In [20]:
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(tfidf_ques)
question_df['cluster'] = clusters

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [21]:
question_df

,title,selftext,body,score_x,urls,cleaned_question,cluster
0,I'm cycling 4000 miles this summer to raise mo...,I'm doing this mostly for my mom who was diagn...,I love this idea. You should repost this in a...,3.0,http://www.cancer.org/Treatment/SupportProgram...,im mostly mom diagnosed breast cancer septembe...,4
1,How did your hair grow back after chemotherapy?,I am just curious.... what happened to your ha...,"Mine was originally thick, wavy and red-brown....",2.0,,curious happened hair chemotherapy mine stick ...,4
2,How did your hair grow back after chemotherapy?,I am just curious.... what happened to your ha...,"My mom always had coarse, thick, wavy hair. A...",2.0,,curious happened hair chemotherapy mine stick ...,4
3,33yo Male...,"Hey everyone, I'm a 33yo male, and my left nip...","Honestly, the internet was my worst enemy at t...",2.0,,everyone im 33yo male left nipple felt irritat...,3
4,33yo Male...,"Hey everyone, I'm a 33yo male, and my left nip...","Breast cancer in men is quite common, more tha...",2.0,,everyone im 33yo male left nipple felt irritat...,3
...,...,...,...,...,...,...,...
65547,For those who did delayed bilateral reconstruc...,Hi - new poster (long time lurker) here. I am ...,I had my mastectomy in June of 2021 and Diep i...,10.0,,new poster long time lurker 42 diagnosed febru...,3
65548,For those who did delayed bilateral reconstruc...,Hi - new poster (long time lurker) here. I am ...,I have Natrelle Inspira “gummy bear” implants....,10.0,,new poster long time lurker 42 diagnosed febru...,3
65549,For those who did delayed bilateral reconstruc...,Hi - new poster (long time lurker) here. I am ...,I had a double mastectomy with tissue expander...,10.0,,new poster long time lurker 42 diagnosed febru...,3
65550,For those who did delayed bilateral reconstruc...,Hi - new poster (long time lurker) here. I am ...,My story is similar to yours. I had a right ma...,10.0,,new poster long time lurker 42 diagnosed febru...,3


In [22]:
from bertopic import BERTopic

# Create a DataFrame to store topics and topic names for each body text
body_topic_df = pd.DataFrame()

for cluster in range(num_clusters):
    # Select body texts and their scores in the current cluster
    cluster_data = question_df[question_df['cluster'] == cluster][['body', 'score_x','urls']]

    # Check if there are body texts in this cluster
    if not cluster_data.empty:
        cluster_body_texts = cluster_data['body'].tolist()
        score_x_values = cluster_data['score_x'].tolist()
        urls = cluster_data['urls'].tolist()

        # Initialize BERTopic and perform topic modeling on the body texts of this cluster
        topic_model = BERTopic(language="english")
        topics, probabilities = topic_model.fit_transform(cluster_body_texts)

        # Get topic names
        topic_names = topic_model.get_topic_info()

        # Create a temporary DataFrame for the current cluster's results
        temp_df = pd.DataFrame({
            'Cluster': cluster,
            'BodyText': cluster_body_texts,
            'Topic': topics,
            'TopicName': [topic_names[topic_names['Topic'] == topic]['Name'].values[0] if topic != -1 else 'Outlier' for topic in topics],
            'ScoreX': score_x_values,
            'urls' : urls
        })

        # Append to the main DataFrame
        body_topic_df = pd.concat([body_topic_df, temp_df], ignore_index=True)

# 'body_topic_df' now contains each cluster, the body text, the derived topic, the topic name, and score_x for each text


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [23]:
body_topic_df

,Cluster,BodyText,Topic,TopicName,ScoreX,urls
0,0,Good idea! I am curious to hear from anyone wh...,0,0_to_and_the_my,9.0,
1,0,Let’s give it a try! Hey Olaparib taking frien...,0,0_to_and_the_my,9.0,
2,0,I had my first taxol/carbo/keytruda on 11/9. I...,0,0_to_and_the_my,9.0,
3,0,I just wanna send hugs and good vibes to every...,2,2_good_beautiful_morning_hope,9.0,
4,0,Has anyone had to face skin cancer post surger...,0,0_to_and_the_my,9.0,
...,...,...,...,...,...,...
65406,4,I have been on it since June 2022 and its work...,107,107_flashes_hot_night_effects,8.0,
65407,4,I have been on it for several years now. It he...,107,107_flashes_hot_night_effects,8.0,
65408,4,It was a godsend for my panic attacks. I was p...,107,107_flashes_hot_night_effects,8.0,
65409,4,I started taking venlafaxine for menopause sym...,27,27_flashes_hot_sweats_effexor,8.0,


In [24]:
# Assume body_topic_df is already loaded and preprocessed
# Define a function to search for topics and return top N body texts based on score
def search_topic(topic_name, top_n=10):
    topic_entries = body_topic_df[body_topic_df['TopicName'].str.contains(topic_name, case=False)]
    sorted_entries = topic_entries.sort_values(by='Score', ascending=False)
    return sorted_entries['BodyText'].head(top_n).tolist()

In [25]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

# Make sure you have downloaded the necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punc_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punc_dict)))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [52]:
import numpy as np
import nltk
import string
import random
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB  # Or any other classifier you use
# from joblib import load  # Uncomment if you're loading a saved model and vectorizer

# Convert to lowercase for uniformity in searching
body_topic_df['TopicName'] = body_topic_df['TopicName'].str.lower()
body_topic_df['BodyText'] = body_topic_df['BodyText'].str.lower()

# NLTK downloads
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Lemmatization and normalization functions
lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punc_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punc_dict)))

# Greeting function
greet_inputs = ('hello', 'hi', 'greetings', 'how are you?', 'Hey')
greet_responses = ["hi", "hey", "hey there!", "good", "I am glad you are talking to me!"]

def greet(sentence):
    for word in sentence.split():
        if word.lower() in greet_inputs:
            return random.choice(greet_responses)

# Function for searching topics in body_topic_df and urls
def search_topic(topic_name, top_n=5):
    matching_entries = body_topic_df[body_topic_df['TopicName'].str.contains(topic_name, case=False)]
    return matching_entries['BodyText'].head(top_n).tolist()

# Function to find reference links
def find_reference_links(word):
    filtered_df = body_topic_df[body_topic_df['BodyText'].str.contains(word, case=False, na=False)]
    urls = filtered_df['URLs'].tolist()
    urls = [url for sublist in urls for url in sublist]  # Flatten the list of lists

    if urls:
        return urls
    else:
        return ["There are no reference links."]

farewell_inputs = ['bye', 'goodbye', 'see you', 'later', 'farewell', 'thank you', 'thanks']

# Function to collect URLs related to a topic
def collect_related_urls(topic_name):
    # Filter the DataFrame for entries containing the topic name
    related_entries = body_topic_df[body_topic_df['TopicName'].str.contains(topic_name, case=False)]
    # Get the URLs from these entries
    related_urls = related_entries['urls'].tolist()
    # Remove any empty URL entries
    related_urls = [url for url in related_urls if url.strip() != '']
    return related_urls

# Modify the response function to include URL fetching
def generate_response(user_response):
    global sentence_tokens  # Use the global sentence_tokens list
    user_response = user_response.lower()

    # Searching the user input in TopicName and fetching related URLs
    topic_results = search_topic(user_response)
    topic_urls = collect_related_urls(user_response)
    response_text = ""
    if topic_results:
        response_text += "Top results for the topic '{}':\n{}".format(user_response, '\n'.join(topic_results))
    if topic_urls:
        response_text += "\n\nRelated URLs:\n" + '\n'.join(topic_urls)
    if response_text:
        return response_text

    # If no direct topic match is found, perform a similarity match
    sentence_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sentence_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    sentence_tokens.remove(user_response)  # Remove the user response after processing

    if vals.shape[1] > 1:
        idx = vals.argsort()[0][-2]
        flat = vals.flatten()
        flat.sort()
        req_tfidf = flat[-2]
        if req_tfidf == 0:
            return "I am sorry! I don't understand you."
        else:
            return "Bot: " + sentence_tokens[idx]
    else:
        return "I am sorry! I don't understand you."

chatbot_output = []

farewell_inputs = ('bye', 'goodbye', 'see you', 'later', 'farewell')

# Main chatbot flow with updated end functions
flag = True
print("Hello! How can I help you with?")
chatbot_output = []  # Initialize a list to store chatbot outputs
sentence_tokens = []  # Initialize an empty list for sentence tokens

while flag:
    user_response = input("You: ").lower()

    if any(farewell in user_response for farewell in farewell_inputs):
        bot_response = "Bot: Goodbye! Have a great day."
        chatbot_output.append(bot_response)
        print(bot_response)
        flag = False
    elif user_response in ['thank you', 'thanks']:
        bot_response = "Bot: You are welcome!"
        chatbot_output.append(bot_response)
        print(bot_response)
        flag = False
    else:
        greeting = greet(user_response)
        if greeting is not None:
            bot_response = "Bot: " + greeting
            chatbot_output.append(bot_response)
            print(bot_response)
        else:
            response_text = generate_response(user_response)  # Use the renamed function
            bot_response = "Bot: " + response_text
            chatbot_output.append(bot_response)
            print(bot_response)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Hello! How can I help you with?
You: tamoxifen
Bot: Top results for the topic 'tamoxifen':
is there an alternative to tamoxifen? it doesn't mix with a medicine i am already on. and i'm supposed to go on it for five years. i asked my doc but haven't gotten a reply yet.
i don't think so. if you're premenopausal.
i started taking tamoxifen yesterday and, to be honest, am a bit afraid because of all these horror stories i've heard. i need a few stories of people who do tolerate or have tolerated tamoxifen well. but google is not very helpful there
i took tamoxifen for 7 years without significant side effects. it was much less troublesome than the ai med i’m on now.
hi all. first post in here. i’m 10-months-post-lumpectomy with second surgery due to complications, but got clear margins. i’ve been on tamoxifen since i finished 24 days of radiation last june. 

i’m perimenopausal, but was still regular and even on the tamoxifen. however, my periods became even heavier and the cramping was ago

In [53]:
chatbot_output

["Bot: Top results for the topic 'tamoxifen':\nis there an alternative to tamoxifen? it doesn't mix with a medicine i am already on. and i'm supposed to go on it for five years. i asked my doc but haven't gotten a reply yet.\ni don't think so. if you're premenopausal.\ni started taking tamoxifen yesterday and, to be honest, am a bit afraid because of all these horror stories i've heard. i need a few stories of people who do tolerate or have tolerated tamoxifen well. but google is not very helpful there\ni took tamoxifen for 7 years without significant side effects. it was much less troublesome than the ai med i’m on now.\nhi all. first post in here. i’m 10-months-post-lumpectomy with second surgery due to complications, but got clear margins. i’ve been on tamoxifen since i finished 24 days of radiation last june. \n\ni’m perimenopausal, but was still regular and even on the tamoxifen. however, my periods became even heavier and the cramping was agonizing. also dealing, ofc, with hot fl

In [54]:
# Text preprocessing function from the notebook
def preprocess_data(text):
    # Remove URLs
    text = re.sub(r'http\\S+', '', text)
    # Remove non-ASCII characters
    text = ''.join([char for char in text if ord(char) < 128])
    # Remove special characters and numbers
    text = re.sub(r'[^A-Za-z\\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Join the tokens back into a string
    cleaned_text = ' '.join(tokens)
    return cleaned_text

In [29]:
for response in chatbot_output:
    cleaned_response = preprocess_data(response)
    # Assuming the text is converted to tf-idf vector
    response_vector = vectorizer.transform([cleaned_response])
    predicted_emotion = nb_classifier.predict(response_vector)  # or svm_classifier
    print(f"Emotion: {predicted_emotion[0]}")

NameError: ignored

In [ ]:
for response in chatbot_output:
    cleaned_response = preprocess_data(response)
    response_vector = vectorizer.transform([cleaned_response])
    predicted_emotion = nb_classifier.predict(response_vector)  # or svm_classifier
    print(f"Response: {response}\nPredicted Emotion: {predicted_emotion[0]}\n")


In [55]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
import seaborn as sns
import matplotlib.pyplot as plt

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [56]:
data_bc = pd.read_csv('/content/breast_cancer_merged.csv')
data_emotions = pd.read_csv('/content/tweet_emotions.csv')

In [57]:
data_emotions = data_emotions[data_emotions['sentiment'] != 'enthusiasm']
data_emotions = data_emotions[data_emotions['sentiment'] != 'surprise']
data_emotions = data_emotions[data_emotions['sentiment'] != 'boredom']
data_emotions = data_emotions[data_emotions['sentiment'] != 'fun']


data_emotions['sentiment'] = data_emotions['sentiment'].apply(lambda x: 'Sad' if x in ['sadness', 'worry'] else x)
data_emotions['sentiment'] = data_emotions['sentiment'].apply(lambda x: 'Happy' if x in ['love','happiness'] else x)
data_emotions['sentiment'] = data_emotions['sentiment'].apply(lambda x: 'Anger' if x in ['anger', 'hate'] else x)
data_emotions['sentiment'] = data_emotions['sentiment'].apply(lambda x: 'Neutral' if x in ['empty', 'neutral','relief'] else x)



In [58]:
data_bc = data_bc[['body']]
data_emotions = data_emotions[['content','sentiment']]

In [59]:
# Function to clean text
def preprocess_data(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove non-ASCII characters
    text = ''.join([char for char in text if ord(char) < 128])

    # Remove special characters and numbers
    text = re.sub(r'[^A-Za-z\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Join the tokens back into a string
    cleaned_text = ' '.join(tokens)

    return cleaned_text

In [60]:
# Apply text preprocessing to 'Column3'
data_bc['cleaned_body'] = data_bc['body'].apply(preprocess_data)
data_emotions['cleaned_content'] = data_emotions['content'].apply(preprocess_data)

In [61]:
data_bc

,body,cleaned_body
0,I love this idea. You should repost this in a...,love idea repost another subreddit since isnt ...
1,"Mine was originally thick, wavy and red-brown....",mine originally thick wavy redbrown grew back ...
2,"My mom always had coarse, thick, wavy hair. A...",mom always coarse thick wavy hair chemo treatm...
3,"Honestly, the internet was my worst enemy at t...",honestly internet worst enemy times diagnosis ...
4,"Breast cancer in men is quite common, more tha...",breast cancer men quite common one would think...
...,...,...
65547,I had my mastectomy in June of 2021 and Diep i...,mastectomy june diep march phase last week fat...
65548,I have Natrelle Inspira “gummy bear” implants....,natrelle inspira gummy bear implants option di...
65549,I had a double mastectomy with tissue expander...,double mastectomy tissue expanders months got ...
65550,My story is similar to yours. I had a right ma...,story similar right mastectomy expander chemo ...


In [62]:
data_emotions

,content,sentiment,cleaned_content
0,@tiffanylue i know i was listenin to bad habi...,Neutral,tiffanylue know listenin bad habit earlier sta...
1,Layin n bed with a headache ughhhh...waitin o...,Sad,layin n bed headache ughhhhwaitin call
2,Funeral ceremony...gloomy friday...,Sad,funeral ceremonygloomy friday
4,@dannycastillo We want to trade with someone w...,Neutral,dannycastillo want trade someone houston ticke...
5,Re-pinging @ghostridah14: why didn't you go to...,Sad,repinging ghostridah didnt go prom bc bf didnt...
...,...,...,...
39995,@JohnLloydTaylor,Neutral,johnlloydtaylor
39996,Happy Mothers Day All my love,Happy,happy mothers day love
39997,Happy Mother's Day to all the mommies out ther...,Happy,happy mothers day mommies woman man long youre...
39998,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...,Happy,niariley wassup beautiful follow peep new hit ...


In [63]:

# Split the training data into features (X) and labels (Y)
X_train = data_emotions['cleaned_content']
y_train = data_emotions['sentiment']

# Split the testing data into features (X_test)
X_test = data_bc['cleaned_body']

# Use TF-IDF Vectorizer to convert text data to numerical vectors
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [64]:

# Train a Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

# Predict the categories for the testing data using Naive Bayes
nb_predictions = nb_classifier.predict(X_test_tfidf)

# Add Naive Bayes predictions to the testing dataframe
data_bc['Predicted_Category_NB'] = nb_predictions


In [65]:
# Train a Linear Support Vector Machine (SVM) classifier
svm_classifier = LinearSVC()
svm_classifier.fit(X_train_tfidf, y_train)

# Predict the categories for the testing data using SVM
svm_predictions = svm_classifier.predict(X_test_tfidf)

# Add SVM predictions to the testing dataframe
data_bc['Predicted_Category_SVM'] = svm_predictions



In [66]:
data_bc

,body,cleaned_body,Predicted_Category_NB,Predicted_Category_SVM
0,I love this idea. You should repost this in a...,love idea repost another subreddit since isnt ...,Sad,Sad
1,"Mine was originally thick, wavy and red-brown....",mine originally thick wavy redbrown grew back ...,Sad,Neutral
2,"My mom always had coarse, thick, wavy hair. A...",mom always coarse thick wavy hair chemo treatm...,Neutral,Neutral
3,"Honestly, the internet was my worst enemy at t...",honestly internet worst enemy times diagnosis ...,Sad,Sad
4,"Breast cancer in men is quite common, more tha...",breast cancer men quite common one would think...,Sad,Sad
...,...,...,...,...
65547,I had my mastectomy in June of 2021 and Diep i...,mastectomy june diep march phase last week fat...,Sad,Sad
65548,I have Natrelle Inspira “gummy bear” implants....,natrelle inspira gummy bear implants option di...,Sad,Sad
65549,I had a double mastectomy with tissue expander...,double mastectomy tissue expanders months got ...,Sad,Sad
65550,My story is similar to yours. I had a right ma...,story similar right mastectomy expander chemo ...,Sad,Sad


In [67]:
# Perform emotion analysis on chatbot output
for response in chatbot_output:
    # Preprocess the response
    cleaned_response = preprocess_data(response)
    # Vectorize the preprocessed response
    # Ensure it's in the correct 2D array format for the classifier
    response_vector = vectorizer.transform([cleaned_response])
    # Predict the emotion
    predicted_emotion = svm_classifier.predict(response_vector)
    print(f"Response: {response}\nPredicted Emotion: {predicted_emotion[0]}\n")


Response: Bot: Top results for the topic 'tamoxifen':
is there an alternative to tamoxifen? it doesn't mix with a medicine i am already on. and i'm supposed to go on it for five years. i asked my doc but haven't gotten a reply yet.
i don't think so. if you're premenopausal.
i started taking tamoxifen yesterday and, to be honest, am a bit afraid because of all these horror stories i've heard. i need a few stories of people who do tolerate or have tolerated tamoxifen well. but google is not very helpful there
i took tamoxifen for 7 years without significant side effects. it was much less troublesome than the ai med i’m on now.
hi all. first post in here. i’m 10-months-post-lumpectomy with second surgery due to complications, but got clear margins. i’ve been on tamoxifen since i finished 24 days of radiation last june. 

i’m perimenopausal, but was still regular and even on the tamoxifen. however, my periods became even heavier and the cramping was agonizing. also dealing, ofc, with hot f

In [68]:
# Perform emotion analysis on chatbot output
for response in chatbot_output:
    # Preprocess the response
    cleaned_response = preprocess_data(response)
    # Vectorize the preprocessed response
    # Ensure it's in the correct 2D array format for the classifier
    response_vector = vectorizer.transform([cleaned_response])
    # Predict the emotion
    predicted_emotion = nb_classifier.predict(response_vector)
    print(f"Response: {response}\nPredicted Emotion: {predicted_emotion[0]}\n")


Response: Bot: Top results for the topic 'tamoxifen':
is there an alternative to tamoxifen? it doesn't mix with a medicine i am already on. and i'm supposed to go on it for five years. i asked my doc but haven't gotten a reply yet.
i don't think so. if you're premenopausal.
i started taking tamoxifen yesterday and, to be honest, am a bit afraid because of all these horror stories i've heard. i need a few stories of people who do tolerate or have tolerated tamoxifen well. but google is not very helpful there
i took tamoxifen for 7 years without significant side effects. it was much less troublesome than the ai med i’m on now.
hi all. first post in here. i’m 10-months-post-lumpectomy with second surgery due to complications, but got clear margins. i’ve been on tamoxifen since i finished 24 days of radiation last june. 

i’m perimenopausal, but was still regular and even on the tamoxifen. however, my periods became even heavier and the cramping was agonizing. also dealing, ofc, with hot f